In [1]:
defi_num = 10
path = "./defi_emb" + str(defi_num) + ".txt"
from transformers import BertModel,BertTokenizer
tokenizer = BertTokenizer.from_pretrained("/home/lawson/pretrain/bert-base-cased")
model = BertModel.from_pretrained("/home/lawson/pretrain/bert-base-cased")

In [2]:
'''简单的方式读取文件
'''
def simpleReadXml(pathData,pathLabel):
    import  xml.dom.minidom as dom
    # step1.先从pathLabel 中找出所有是双关语的句子
    allHomo = [] # 标记所有是双关语的句子集合 [hom_1,hom_2]
    allLabel =[] # 找出所有的双关词 [hom_1_12]
    punWord = [] # 表示所有的双关词
    with open(pathLabel,'r') as f:
        line = f.readline()
        while (line!=""):
            line = line.strip() # 去行末换行
            line = line.split() # 空格分割            
            #print(line[0])
            allHomo.append(line[0])
            allLabel.append(line[1]) # 将双关词的id放入其中
            line = f.readline()
    #print(allHomo)
    
    # step2.接着读取双关句，成为一行文本
    #打开xml文档
    dom2 = dom.parse(pathData)
    #得到文档元素对象
    root = dom2.documentElement    
    texts = root.getElementsByTagName("text") # 得到所有的text             
    puns = [] # 存储双关语的列表
    for text in texts:
        name = text.getAttribute('id')
        if name in allHomo:
            words = text.getElementsByTagName("word") #得到word
            pun = []
            for word in words:
                a = word.firstChild.data                
                pun.append(a)
                if word.getAttribute('id') in allLabel:
                    punWord.append(a)  # 这个单词就是双关词
            puns.append(pun)
    
    return puns,punWord  # 1607条语义双关语 以及 对应的双关词

In [3]:
# 最后运行的模块
pathData = "/home/lawson/program/data/puns/test/homo/subtask2-homographic-test.xml"
pathLabel = "/home/lawson/program/data/puns/test/homo/subtask2-homographic-test.gold"
puns,punWord = simpleReadXml(pathData,pathLabel)

In [4]:
wordMap = {} # 找出所有的单词 word => list
for pun in puns:
    for word in pun: # 找出所有的单词
        if word not in wordMap.keys():
            wordMap[word] = []
        else:
            continue
print(len(wordMap)) # 有4567 个词

4567


In [5]:
words = list(wordMap.keys()) # 转为list
from nltk.corpus import wordnet as wn
import random
# step1.获取所有单词
for word in words:
    senses = wn.synsets(word)
    gross = []
    for _ in senses:        
        gross.append(_.definition())
    print(gross)
        
    # step2.处理定义    
    while(len(gross) > defi_num): # 如果超过一定的设置，则删除
        index = int(random.random() * defi_num)
        del(gross[index]) # 删除index 上的值        
        
    if len(gross)<1:
        val = word + " None\n"
        with open(path,'a') as f:
            f.write(val)
    else:    
        inputs = tokenizer(gross,
                      padding='max_length',
                      truncation=True,
                       max_length=40,
                       return_tensors='pt')
        out = model(**inputs)
        last_layer = out.last_hidden_state 
        cls_emb = last_layer[:,0,:] # 只取 CLS 上的值

        # step3.将结果写入到文件中
        cls_emb = cls_emb.tolist()
        with open(path,'a') as f: # 写入单词
            f.write(word+" ")
        for emb in cls_emb:    
            res = ""
            for data in emb:
                #print(data)
                res += (str(data)) 
                res += " "
            res.strip(" ")
            res+="\n"
            with open(path,'a') as f:
                f.write(res)

[]
['prevent from being seen or discovered', 'be or go into hiding; keep out of sight, as for protection and safety', 'cover as if with a shroud', 'make undecipherable or imperceptible by obscuring or concealing']
[]
[]
['a professional killer who uses a gun', 'a person who shoots a gun (as regards their ability)']
['a unit of length equal to one twelfth of a foot', 'a rare soft silvery metallic element; occurs in small quantities in sphalerite', 'a state in midwestern United States', 'holding office', 'directed or bound inward', 'currently fashionable', 'to or toward the inside of']
['a metric unit of length equal to one ten billionth of a meter (or 0.0001 micron); used to specify wavelengths of electromagnetic radiation', 'any of several fat-soluble vitamins essential for normal vision; prevents night blindness or inflammation or dryness of the eyes', 'one of the four nucleotides used in building DNA; all four nucleotides have a common phosphate group and a sugar (ribose)', '(biochem

['the state of being in effect or being operative', 'a business especially one run on a large scale', 'a planned activity involving many people performing various actions', '(computer science) data processing in which the result is completely specified by a rule (especially the processing that results from a single instruction)', 'activity by a military or naval force (as a maneuver or campaign)', 'a medical procedure involving an incision with instruments; performed to repair damage or arrest disease in a living body', 'a process or series of acts especially of a practical or mechanical nature involved in a particular form of work', 'process or manner of functioning or operating', '(psychology) the performance of some composite cognitive activity; an operation that affects mental contents', '(mathematics) calculation by mathematical methods', 'the activity of operating something (a machine or business etc.)']
[]
['a metric unit of length equal to one ten billionth of a meter (or 0.000

['sloping land (especially the slope beside a body of water)', 'a financial institution that accepts deposits and channels the money into lending activities', 'a long ridge or pile', 'an arrangement of similar objects in a row or in tiers', 'a supply or stock held in reserve for future use (especially in emergencies)', 'the funds held by a gambling house or the dealer in some gambling games', 'a slope in the turn of a road or track; the outside is higher than the inside in order to reduce the effects of centrifugal force', 'a container (usually with a slot in the top) for keeping money at home', 'a building in which the business of banking transacted', 'a flight maneuver; aircraft tips laterally about its longitudinal axis (especially in turning)', 'tip laterally', 'enclose with a bank', 'do business with a bank or keep an account at a bank', 'act as the banker in a game or in gambling', 'be in the banking business', 'put into a bank account', 'cover with ashes so to control the rate o

[]
['on one occasion', 'as soon as', 'at a previous time']
['water frozen in the solid state', 'the frozen part of a body of water', 'diamonds', 'a flavored sugar topping used to coat and decorate cakes', 'a frozen dessert with fruit flavoring (especially one containing no milk)', 'an amphetamine derivative (trade name Methedrine) used in the form of a crystalline hydrochloride; used as a stimulant to the nervous system and as an appetite suppressant', 'a heat engine in which combustion occurs inside the engine rather than in a separate furnace; heat expands a gas that either moves a piston or turns a gas turbine', 'a rink with a floor of ice for ice hockey or ice skating', 'decorate with frosting', 'cause to become ice or icy', 'put ice on or put on ice']
['the best people or things in a group', 'the part of milk containing the butterfat', 'toiletry consisting of any of various substances in the form of a thick liquid that have a soothing and moisturizing effect when applied to the sk

['the pure mathematics of points and lines and curves and surfaces']
['a collection of things sharing a common attribute', 'a body of students who are taught together', 'people having the same social, economic, or educational status', 'education imparted in a series of lessons or meetings', 'a league ranked by quality', 'a body of students who graduate together', '(biology) a taxonomic group containing one or more orders', 'elegance in dress or behavior', 'arrange or order by classes or categories']
['have the quality of being; (copula, used with an adjective or a predicate noun)', 'be identical to; be someone or something', 'occupy a certain position or area; be somewhere', 'have an existence, be extant', 'happen, occur, take place', 'be identical or equivalent to', 'form or compose', 'work in a specific place, with a specific subject, or in a specific function', 'represent, as of a character on stage', 'spend or use time', 'have life, be alive', 'to remain unmolested, undisturbed, or

...

['uncastrated adult male horse', 'constituting the full quantity or extent; complete', 'constituting the undiminished entirety; lacking nothing essential especially not damaged; - Bacon', '(of leaves or petals) having a smooth edge; not broken up into teeth or lobes', '(used of domestic animals) sexually competent']
[]
['one of the twelve Apostles (first century)', 'United States singer and songwriter (born in 1942)', 'United States playwright noted for light comedies (born in 1927)', 'United States economist and psychologist who pioneered in the development of cognitive science (1916-2001)']
['not successful; having failed or having an unfavorable outcome', 'failing to accomplish an intended result']
['a subdivision of a written work; usually numbered and titled', "any distinct period in history or in a person's life", 'a local branch of some fraternity or association', 'an ecclesiastical assembly of the monks in a monastery or even of the canons of a church', 'a series of related eve

['a reference book that is published regularly once every year', 'occurring or payable every year', 'without missing a year']
['a vessel containing liquid in which something is immersed (as to process it or to maintain it at a constant temperature or to lubricate it)', 'you soak and wash your body in a bathtub', 'a relatively large open container that you fill with water and use to wash the body', 'an ancient Hebrew liquid measure equal to about 10 gallons', 'a town in southwestern England on the River Avon; famous for its hot springs and Roman remains', 'a room (as in a residence) containing a bathtub or shower and usually a washbasin and toilet', "clean one's body by immersion into water"]
['the month following May and preceding July']
['a prominent attribute or aspect of something', 'a distinguishing quality', 'the integer part (positive or negative) of the representation of a logarithm; in the expression log 643 = 2.808 the characteristic is 2', 'any measurable property of a device

['motor that converts thermal energy to mechanical work', 'something used to achieve a purpose', 'a wheeled vehicle consisting of a self-propelled engine that is used to draw trains along railway tracks', 'an instrument or machine that is used in warfare, such as a battering ram, catapult, artillery piece, etc.']
['German physicist who invented the mercury thermometer and developed the scale of temperature that bears his name (1686-1736)', 'of or relating to a temperature scale proposed by the inventor of the mercury thermometer']
['find the solution to (a problem or question) or understand the meaning of', 'find the solution', 'settle, as of a debt', 'explained or answered']
['a position on a scale of intensity or amount or quality', 'a specific identifiable position in a continuum or series or especially in a process', 'an award conferred by a college or university signifying that the recipient has satisfactorily completed a course of study', 'a measure for arcs and angles', 'the hig

['an expert who gives advice']
['(of businesses and businessmen) unscrupulous', 'of questionable taste or morality', 'not as expected', 'filled with shade']
['in large part; mainly or chiefly', 'usually; as a rule']
['worthless material that is to be disposed of', 'worthless people', 'nonsensical talk or writing', 'an amphetamine derivative (trade name Methedrine) used in the form of a crystalline hydrochloride; used as a stimulant to the nervous system and as an appetite suppressant', 'dispose of (something useless or old)', 'express a totally negative opinion of']
['stay clear from; keep away from; keep out of the way of someone or something', 'prevent the occurrence of; prevent from happening', 'refrain from doing something', 'refrain from certain foods or beverages', 'declare invalid']
['plants with sword-shaped leaves and erect stalks bearing bright-colored flowers composed of three petals and three drooping sepals', 'muscular diaphragm that controls the size of the pupil which in

['proceed somewhere despite the risk of possible dangers', 'put forward, of a guess, in spite of possible refutation', 'put at risk']
['a person who requires medical care', 'the semantic role of an entity that is not the agent but is directly involved in or affected by the happening denoted by the verb in the clause']
['get something; come into possession of', 'receive a specified treatment (abstract)', 'register (perceptual input)', 'go through (mental or physical states or experiences)', "express willingness to have in one's home or environs", 'accept as true or valid', 'bid welcome to; greet upon arrival', 'convert into sounds or pictures', 'experience as a reaction', 'have or give a reception', 'receive as a retribution or punishment', 'partake of the Holy Eucharist sacrament', 'regard favorably or with disapproval']
['(surgery) tissue or organ transplanted from a donor to a recipient; in some cases the patient can be both donor and recipient', 'an operation moving an organ from on

In [8]:
"""检测生成的数据
"""
wordEmb={}# str => list
with open(path,'r') as f:
    line = f.readline()    
    while(line): # 只要有内容，就要尝试                
        print(line)
        line = line.split() # 先按照空格分割
        if line[-1] == "None": # 以None结尾
            pass            
        elif line[0].isalpha(): # 如果是字符。是一个新的开始            
            res1 = line[0] # 得到单词
            wordEmb[res1] = emb
            emb = [] # 装下当下单词所有的emb            
            del(line[0])
            line = [float(_) for _ in line] # 全部转为float 型
            emb.append(line)            
        else:            
            line = [float(_) for _ in line] # 全部转为float 型
            print(line)
            emb.append(line)
        line = f.readline()
print(wordEmb[0])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.5166175365447998 -0.25477954745292664 -0.17960785329341888 -0.474720299243927 -0.08959367871284485 -0.03384559974074364 0.04943522438406944 0.036459289491176605 0.3048180341720581 -1.1875742673873901 -0.2553043067455292 0.1903444230556488 -0.1234070360660553 -0.4174104332923889 -0.20591074228286743 0.1779012531042099 0.2637081444263458 -0.09562701731920242 0.002606905298307538 -0.11296070367097855 0.18214330077171326 -0.1410709172487259 0.8247764706611633 -0.24859830737113953 -0.05880296602845192 -0.11381886154413223 -0.2774946987628937 0.01263192668557167 -0.23118051886558533 0.13077068328857422 -0.043373920023441315 0.23984888195991516 -0.2752543091773987 -0.0062843733467161655 -0.06072697043418884 -0.1460038274526596 0.06246434524655342 -0.02503995969891548 0.004877124447375536 0.10194455087184906 -0.2918524146080017 0.021273741498589516 0.5972208976745605 0.004194406792521477 0.1898292601108551 -0.15694500505924225 0.14902423322200775 0.19283653795719147 -0.08258496969938278 0.11

KeyError: 0